# 🤖 TARS v3 — Google Colab Full Training Pipeline

**Полное обучение ТАРС v3: мозг + голос (STT + TTS)**

### Фазы обучения:
| # | Модуль | Компоненты | Время (T4) |
|---|--------|------------|------------|
| 2 | Рефлексы | MinGRU Classifier | ~2 мин |
| 3 | MinGRU LM | System 1 (быстрая генерация) | ~30 мин |
| 4 | Mamba-2 Brain | SSD + WKV + Ω-SSM + MoLE + WaveConsolidation | ~3-5 ч |
| 5 | Квантизация 1.58-bit | BitNet STE fine-tune (5 эпох) | ~20 мин |
| 7 | Валидация | Тестовая генерация | ~5 мин |
| 8 | Whisper STT | LoRA fine-tune (base, русский) | ~40 мин |
| 9 | Piper TTS | VITS fine-tune (русский голос) | ~1-2 ч |
| 10 | Voice INT8 | Квантизация ONNX + Whisper Boost | ~5 мин |

### Инструкции:
1. **Runtime → Change runtime type → GPU (T4)**
2. Запускай ячейки по порядку ▶
3. Модели автоматически сохраняются на **Google Drive**
4. При отключении Colab — запусти ячейку 0 и нужную фазу заново (модели восстановятся с Drive)

---

## 0. Проверка GPU + Google Drive + Восстановление моделей

In [ ]:
import torch
import os
import time
import shutil

print("=" * 65)
print("  TARS v3 — Google Colab Full Training Pipeline")
print("=" * 65)

if torch.cuda.is_available():
    GPU_NAME = torch.cuda.get_device_name(0)
    VRAM_GB = torch.cuda.get_device_properties(0).total_mem / 1024**3
    print(f"  ✅ GPU:  {GPU_NAME}")
    print(f"  ✅ VRAM: {VRAM_GB:.1f} GB")
    print(f"  ✅ CUDA: {torch.version.cuda}")
else:
    print("  ❌ GPU НЕ ОБНАРУЖЕН!")
    print("  → Runtime → Change runtime type → GPU (T4)")
    raise RuntimeError("GPU not available")

# Адаптивные параметры по GPU
if VRAM_GB >= 35:     # A100 (40GB)
    BATCH, ACCUM = 32, 2
    WHISPER_BATCH = 32
    WHISPER_MODEL = "base"   # 74M params — лучше качество
    PIPER_BATCH = 32
    GPU_TIER = "A100"
elif VRAM_GB >= 14:   # T4 (16GB)
    BATCH, ACCUM = 16, 4
    WHISPER_BATCH = 16
    WHISPER_MODEL = "base"   # 74M — помещается на T4
    PIPER_BATCH = 16
    GPU_TIER = "T4"
else:                 # K80 / free
    BATCH, ACCUM = 8, 8
    WHISPER_BATCH = 8
    WHISPER_MODEL = "tiny"   # 39M — для слабых GPU
    PIPER_BATCH = 8
    GPU_TIER = "Budget"

print(f"  ⚡ Tier: {GPU_TIER} (batch={BATCH}×{ACCUM}={BATCH*ACCUM})")
print(f"  🎤 Whisper: {WHISPER_MODEL}")

# Google Drive — автосохранение чекпоинтов
SAVE_TO_DRIVE = True
DRIVE_DIR = "/content/drive/MyDrive/TARS_v3_models"

if SAVE_TO_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    os.makedirs(DRIVE_DIR, exist_ok=True)
    print(f"  💾 Google Drive: {DRIVE_DIR}")

print("=" * 65)

T_START = time.time()

## 1. Клонирование репозитория + Восстановление с Drive

In [ ]:
import os, sys, shutil, glob

REPO_URL = "https://github.com/Vazilll/TarsSSM-Py.git"
WORK_DIR = "/content/TarsSSM-Py"

if os.path.exists(os.path.join(WORK_DIR, "mega_train.py")):
    print("✅ Репозиторий уже клонирован")
    !cd {WORK_DIR} && git pull --ff-only 2>/dev/null || true
else:
    print(f"📥 Клонирование {REPO_URL}...")
    !git clone {REPO_URL} {WORK_DIR}

os.chdir(WORK_DIR)
sys.path.insert(0, WORK_DIR)

# Установка зависимостей
!pip install -q einops tqdm psutil sentencepiece tokenizers \
    sentence-transformers datasets transformers peft jiwer \
    faster-whisper onnxruntime sounddevice

print(f"\n📂 Рабочая директория: {os.getcwd()}")
print("✅ Зависимости установлены")

# ═══ ВОССТАНОВЛЕНИЕ МОДЕЛЕЙ С DRIVE ═══
# Если Colab отключился — модели уже на Drive, восстанавливаем
if SAVE_TO_DRIVE and os.path.exists(DRIVE_DIR):
    restore_map = {
        # Drive file → Local path
        "mamba2_final.pt":   "models/mamba2/mamba2_omega.pt",
        "mamba2_phase3.pt": "models/mamba2/mamba2_omega.pt",
        "mamba2_phase2.pt": "models/mamba2/mamba2_omega.pt",
        "mamba2_phase1.pt": "models/mamba2/mamba2_omega.pt",
        "mamba2_158bit.pt": "models/mamba2/mamba2_omega_158bit.pt",
        "mingru.pt":        "models/mingru_weights.pt",
        "reflex.pt":        "models/reflex/reflex_classifier.pt",
    }
    restored = 0
    for drive_name, local_path in restore_map.items():
        drive_path = os.path.join(DRIVE_DIR, drive_name)
        local_full = os.path.join(WORK_DIR, local_path)
        if os.path.exists(drive_path) and not os.path.exists(local_full):
            os.makedirs(os.path.dirname(local_full), exist_ok=True)
            shutil.copy2(drive_path, local_full)
            size_mb = os.path.getsize(local_full) / 1024 / 1024
            print(f"  🔄 Drive → {local_path} ({size_mb:.1f} MB)")
            restored += 1
    # Whisper LoRA directory
    drive_whisper = os.path.join(DRIVE_DIR, "whisper_ru_lora")
    local_whisper = os.path.join(WORK_DIR, "models", "voice", "whisper_ru_lora")
    if os.path.exists(drive_whisper) and not os.path.exists(local_whisper):
        shutil.copytree(drive_whisper, local_whisper)
        print(f"  🔄 Drive → whisper_ru_lora/")
        restored += 1
    if restored > 0:
        print(f"  ✅ Восстановлено {restored} моделей с Google Drive")
        print(f"  → Можно пропустить уже обученные фазы!")
    else:
        print("  ℹ На Drive нет предыдущих моделей — обучение с нуля")

# Утилита для автосохранения на Drive
def save_to_drive(src_path, name=None):
    """Копирует файл на Google Drive для безопасности."""
    if not SAVE_TO_DRIVE or not os.path.exists(src_path):
        return
    dst = os.path.join(DRIVE_DIR, name or os.path.basename(src_path))
    shutil.copy2(src_path, dst)
    size_mb = os.path.getsize(dst) / 1024 / 1024
    print(f"  💾 → Drive: {name or os.path.basename(src_path)} ({size_mb:.1f} MB)")

## 2. Скачивание обучающих данных

In [ ]:
import glob
os.makedirs("data", exist_ok=True)

# Wikipedia (50к статей)
wiki_path = os.path.join(WORK_DIR, "data", "wiki_ru.txt")
if os.path.exists(wiki_path) and os.path.getsize(wiki_path) > 100_000:
    size_mb = os.path.getsize(wiki_path) / 1024 / 1024
    print(f"📚 Wikipedia: уже есть ({size_mb:.1f} MB)")
else:
    print("📚 Скачивание Wikipedia (50 000 статей)...")
    !python training/download_wiki.py --count 50000

# HuggingFace datasets
hf_files = glob.glob(os.path.join(WORK_DIR, "data", "hf_*.txt"))
if len(hf_files) >= 1:
    total_mb = sum(os.path.getsize(f) for f in hf_files) / 1024 / 1024
    print(f"🤗 HuggingFace: уже есть ({len(hf_files)} файлов, {total_mb:.0f} MB)")
else:
    print("🤗 Скачивание HuggingFace датасетов...")
    !python training/download_hf_dataset.py --preset all

# LEANN embedding model
try:
    emb_path = os.path.join(WORK_DIR, "models", "embeddings")
    if not os.path.exists(emb_path):
        from sentence_transformers import SentenceTransformer
        model = SentenceTransformer('all-MiniLM-L6-v2')
        model.save(emb_path)
    print(f"🧠 Embedding model: {emb_path}")
except Exception as e:
    print(f"  ⚠ Embeddings: {e}")

# LEANN ingest
try:
    sys.path.insert(0, os.path.join(WORK_DIR, "training"))
    from ingest_to_leann import ingest_all
    ingest_all()
    print("🧠 LEANN заполнена")
except Exception as e:
    print(f"  ℹ LEANN: {e} (не критично)")

data_files = [f for f in glob.glob(os.path.join(WORK_DIR, "data", "*")) if os.path.isfile(f)]
total = sum(os.path.getsize(f) for f in data_files)
print(f"\n📊 Итого данных: {total / 1024 / 1024:.0f} MB")

---
# 🧠 МОЗГ — Фазы 2-7
---

## Фаза 2: Рефлексы (MinGRU Classifier)

~2 мин на GPU

In [ ]:
print("=" * 65)
print("  Фаза 2: Рефлексы (MinGRU Classifier)")
print("=" * 65)
!python training/train_reflex.py --epochs 100 --lr 0.002
save_to_drive("models/reflex/reflex_classifier.pt", "reflex.pt")

## Фаза 3: MinGRU Language Model (System 1)

Обучаемые модули: **MinGRU cells, embedding, LM head, Ω-SSM context**

~30 мин на T4

In [ ]:
print("=" * 65)
print("  Фаза 3: MinGRU Language Model (System 1)")
print("=" * 65)
!python training/train_mingru.py \
    --epochs 50 \
    --lr 3e-3 \
    --dim 512 \
    --layers 6 \
    --batch 32 \
    --seq_len 256 \
    --augment
save_to_drive("models/mingru_weights.pt", "mingru.pt")

## Фаза 4: Mamba-2 Brain (12×768d — 4 под-фазы)

Обучаемые модули по фазам:
- **Phase 1** (8 эпох): ВСЕ — SSD, WKV, Ω-SSM, MoLE, WaveConsolidation
- **Phase 2** (5 эпох): WKV + Fusion + Ω-SSM + MoLE + NoveltyGate (SSD frozen)
- **Phase 3** (3 эпохи): MoLE routing + MatrixPool + WaveConsolidation + NoveltyGate
- **Phase 4** (3 эпохи): WKV + RAG + Memory injection + Spine

~3-5 часов на T4

In [ ]:
import torch, os

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"GPU: {torch.cuda.get_device_name(0)} ({VRAM_GB:.0f} GB)")
print(f"Batch: {BATCH} × {ACCUM} = {BATCH * ACCUM} effective")

# Transfer embedding MinGRU → Mamba-2
emb_args = ""
mingru_path = os.path.join(WORK_DIR, "models", "mingru_weights.pt")
if os.path.exists(mingru_path):
    cp = torch.load(mingru_path, map_location='cpu', weights_only=False)
    state = cp.get('model_state_dict', cp)
    for k in state:
        if 'shared_embedding' in k or 'emb.weight' in k:
            emb_dir = os.path.join(WORK_DIR, "models", "tars_v3")
            os.makedirs(emb_dir, exist_ok=True)
            emb_path = os.path.join(emb_dir, "_transfer_embedding.pt")
            torch.save(state[k], emb_path)
            emb_args = f"--pretrained_emb {emb_path}"
            print(f"🔗 Transfer embedding: {state[k].shape}")
            break

BASE = f"--d_model 768 --n_layers 12 --vocab_size 256 --batch {BATCH} --accum_steps {ACCUM} --device {device} --curriculum --label_smoothing 0.1 {emb_args}"

In [ ]:
# Phase 1/4: Full pretrain — ВСЕ компоненты (8 эпох)
print("=" * 65)
print("  Phase 1/4: Full pretrain (SSD + WKV + Ω-SSM + MoLE + WaveConsolidation)")
print("=" * 65)
!python training/train_mamba2.py {BASE} --epochs 8 --lr 3e-4 --phase 1 --seq_len 256
save_to_drive("models/mamba2/mamba2_omega.pt", "mamba2_phase1.pt")

In [ ]:
# Phase 2/4: WKV + Fusion + Ω-SSM + NoveltyGate (SSD frozen, 5 эпох)
print("=" * 65)
print("  Phase 2/4: Fine-tune WKV + Fusion + Ω-SSM + NoveltyGate")
print("=" * 65)
!python training/train_mamba2.py {BASE} --epochs 5 --lr 1e-4 --phase 2 --seq_len 512 --resume
save_to_drive("models/mamba2/mamba2_omega.pt", "mamba2_phase2.pt")

In [ ]:
# Phase 3/4: MoLE + MatrixPool + WaveConsolidation + NoveltyGate (3 эпохи)
print("=" * 65)
print("  Phase 3/4: Fine-tune MoLE + MatrixPool + WaveMerge + NoveltyGate")
print("=" * 65)
!python training/train_mamba2.py {BASE} --epochs 3 --lr 3e-5 --phase 3 --seq_len 512 --resume
save_to_drive("models/mamba2/mamba2_omega.pt", "mamba2_phase3.pt")

In [ ]:
# Phase 4/4: WKV + RAG + Memory + Spine (3 эпохи)
print("=" * 65)
print("  Phase 4/4: Fine-tune WKV + RAG + Memory + to/from_memory_space")
print("=" * 65)
!python training/train_mamba2.py {BASE} --epochs 3 --lr 1.5e-5 --phase 4 --seq_len 512 --resume
save_to_drive("models/mamba2/mamba2_omega.pt", "mamba2_final.pt")

## Фаза 5: Квантизация BitNet 1.58-bit (5 эпох STE)

FP16 → 1.58-bit (260 MB → ~60 MB). STE (Straight-Through Estimator) обучает квантованную модель.

**Увеличено с 3→5 эпох** — модель лучше адаптируется к 1.58-bit весам.

~20 мин

In [ ]:
fp16_path = os.path.join(WORK_DIR, "models", "mamba2", "mamba2_omega.pt")

if os.path.exists(fp16_path):
    fp16_mb = os.path.getsize(fp16_path) / 1024 / 1024
    print("=" * 65)
    print(f"  Фаза 5: Квантизация BitNet 1.58-bit (FP16: {fp16_mb:.0f} MB)")
    print("=" * 65)
    !python training/train_mamba2.py \
        --d_model 768 --n_layers 12 \
        --batch {BATCH} --accum_steps {ACCUM} \
        --epochs 5 --lr 5e-5 \
        --phase 1 --quant \
        --resume --device cuda \
        --seq_len 256 --label_smoothing 0.1
    # Сохраняем И FP16 и 158bit на Drive
    save_to_drive("models/mamba2/mamba2_omega.pt", "mamba2_fp16.pt")
    save_to_drive("models/mamba2/mamba2_omega_158bit.pt", "mamba2_158bit.pt")
else:
    print("⚠ FP16 модель не найдена, пропускаем квантизацию")

## Фаза 7: Валидация мозга

In [ ]:
import torch, sys
sys.path.insert(0, WORK_DIR)

print("=" * 65)
print("  Фаза 7: Валидация мозга")
print("=" * 65)

try:
    from brain.tokenizer import TarsTokenizer
    from brain.mamba2.model import TarsMamba2LM

    device = "cuda" if torch.cuda.is_available() else "cpu"
    tokenizer = TarsTokenizer()
    model, ckpt = TarsMamba2LM.load_pretrained(device=device)
    model.eval()

    if ckpt is None:
        print("⚠ Trained weights not found")
    else:
        params = sum(p.numel() for p in model.parameters())
        print(f"✅ Model: {ckpt}")
        print(f"✅ Parameters: {params:,}")
        print()

        for prompt in ["привет", "как дела", "что такое", "расскажи", "помоги"]:
            tokens = tokenizer.encode(prompt)
            input_ids = torch.tensor([tokens], dtype=torch.long, device=device)
            with torch.no_grad():
                logits = model(input_ids)
            probs = torch.softmax(logits[0, -1, :], dim=-1)
            top5 = torch.topk(probs, 5)
            preds = []
            for idx, prob in zip(top5.indices.tolist(), top5.values.tolist()):
                char = tokenizer.decode([idx])
                preds.append(f"'{char}'({prob:.2%})")
            print(f"  '{prompt}' → {', '.join(preds)}")

        print("\n✅ Brain works!")
    del model
    torch.cuda.empty_cache()
except Exception as e:
    print(f"❌ Validation error: {e}")

---
# 🎤 ГОЛОС — Фазы 8-10
---

## Фаза 8: Whisper STT — Распознавание речи (LoRA)

Fine-tune **Whisper base** (74M) на русском через LoRA адаптеры.

**Whisper base** — 2× лучше чем tiny для русского языка.

Данные: **Common Voice Russian** (10K примеров, 5 эпох)

~40 мин на T4

In [ ]:
print("=" * 65)
print(f"  Фаза 8: Whisper {WHISPER_MODEL} LoRA Fine-tune (Russian STT)")
print("=" * 65)

import gc, torch
gc.collect()
torch.cuda.empty_cache()

!python training/train_whisper.py \
    --model {WHISPER_MODEL} \
    --device cuda \
    --samples 10000 \
    --val_samples 1000 \
    --epochs 5 \
    --batch {WHISPER_BATCH} \
    --lr 1e-3 \
    --lora_r 32

# Сохранение LoRA адаптеров на Drive
whisper_dir = os.path.join(WORK_DIR, "models", "voice", "whisper_ru_lora")
if os.path.exists(whisper_dir):
    drive_whisper = os.path.join(DRIVE_DIR, "whisper_ru_lora") if SAVE_TO_DRIVE else None
    if drive_whisper:
        import shutil
        if os.path.exists(drive_whisper):
            shutil.rmtree(drive_whisper)
        shutil.copytree(whisper_dir, drive_whisper)
        print(f"  💾 → Drive: whisper_ru_lora/")
    print("✅ Whisper STT обучен!")
else:
    print("⚠ Whisper LoRA не найден")

## Фаза 9: Piper TTS — Синтез речи (VITS)

Fine-tune **Piper TTS** на RUSLAN (31ч русской речи).

~1-2 часа на T4

In [ ]:
print("=" * 65)
print("  Фаза 9: Piper TTS Fine-tune (Russian Voice)")
print("=" * 65)

gc.collect()
torch.cuda.empty_cache()

!pip install -q piper-tts piper-phonemize 2>/dev/null || echo "piper install skipped"

!python training/train_piper.py \
    --epochs 1000 \
    --max_samples 3000 \
    --batch {PIPER_BATCH}

piper_onnx = os.path.join(WORK_DIR, "models", "voice", "tars_voice_ru.onnx")
if os.path.exists(piper_onnx):
    save_to_drive(piper_onnx, "tars_voice_ru.onnx")
    print("✅ Piper TTS обучен!")
else:
    print("⚠ Piper ONNX не создан")

## Фаза 10: Whisper Boost + Voice INT8 Квантизация

~5 мин

In [ ]:
print("=" * 65)
print("  Фаза 10: Whisper Boost + Voice INT8 Квантизация")
print("=" * 65)

print("\n📝 Whisper Boost: извлечение ключевых слов...")
!python training/whisper_boost.py

whisper_ctx = os.path.join(WORK_DIR, "models", "voice", "whisper_context.json")
if os.path.exists(whisper_ctx):
    save_to_drive(whisper_ctx, "whisper_context.json")

print("\n🔧 INT8 квантизация голосовых моделей...")
!python training/quantize_voice.py

print("\n✅ Голосовой пайплайн завершён!")

---
# 📦 Финальная сборка + Скачивание
---

In [ ]:
import shutil, json, time

OUTPUT_DIR = "/content/tars_v3_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("=" * 65)
print("  Финальная сборка tars_v3")
print("=" * 65)

# Все модели (FP16 + 158bit + MinGRU + Reflex + Voice)
model_files = {
    "mamba2.pt":        os.path.join(WORK_DIR, "models", "mamba2", "mamba2_omega.pt"),
    "mamba2_158bit.pt": os.path.join(WORK_DIR, "models", "mamba2", "mamba2_omega_158bit.pt"),
    "mingru.pt":        os.path.join(WORK_DIR, "models", "mingru_weights.pt"),
    "reflex.pt":        os.path.join(WORK_DIR, "models", "reflex", "reflex_classifier.pt"),
    "tars_voice_ru.onnx": os.path.join(WORK_DIR, "models", "voice", "tars_voice_ru.onnx"),
    "whisper_context.json": os.path.join(WORK_DIR, "models", "voice", "whisper_context.json"),
}

print("\n📦 Модели:")
saved_models = []
for name, src in model_files.items():
    if os.path.exists(src):
        dst = os.path.join(OUTPUT_DIR, name)
        shutil.copy2(src, dst)
        size_mb = os.path.getsize(dst) / 1024 / 1024
        print(f"  ✅ {name} ({size_mb:.1f} MB)")
        saved_models.append(name)
        # Дополнительная копия на Drive!
        save_to_drive(src, name)
    else:
        print(f"  ⏭ {name} — не найден")

# Whisper LoRA
whisper_lora_src = os.path.join(WORK_DIR, "models", "voice", "whisper_ru_lora")
whisper_lora_dst = os.path.join(OUTPUT_DIR, "whisper_ru_lora")
if os.path.exists(whisper_lora_src):
    if os.path.exists(whisper_lora_dst):
        shutil.rmtree(whisper_lora_dst)
    shutil.copytree(whisper_lora_src, whisper_lora_dst)
    print(f"  ✅ whisper_ru_lora/ (LoRA адаптеры)")
    saved_models.append("whisper_ru_lora/")

# Config
config = {
    "version": "3.0",
    "models": {
        "mamba2": {
            "params": {
                "encoding": "cp1251", "vocab_size": 256,
                "d_model": 768, "n_layers": 12,
                "n_experts": 8, "omega_dim": 32, "pool_size": 48
            }
        },
        "mingru": {"dim": 512, "layers": 6},
        "voice": {
            "stt": f"whisper-{WHISPER_MODEL} + LoRA (Russian)",
            "tts": "piper-vits (Russian)",
            "boost": "whisper_context.json"
        }
    },
    "trained_on": "Google Colab",
    "gpu": GPU_NAME,
}
with open(os.path.join(OUTPUT_DIR, "config.json"), "w") as f:
    json.dump(config, f, ensure_ascii=False, indent=2)

# Архив
archive_path = "/content/tars_v3_full"
shutil.make_archive(archive_path, 'gztar', OUTPUT_DIR)
archive_file = archive_path + ".tar.gz"
size_mb = os.path.getsize(archive_file) / 1024 / 1024

if SAVE_TO_DRIVE:
    shutil.copy2(archive_file, os.path.join(DRIVE_DIR, "tars_v3_full.tar.gz"))
    print(f"\n💾 Архив сохранён на Google Drive!")

total_time = time.time() - T_START
hours = total_time / 3600

print(f"\n{'=' * 65}")
print(f"  ИТОГИ ОБУЧЕНИЯ")
print(f"{'=' * 65}")
print(f"  ⏱  Время: {hours:.1f} часов ({total_time:.0f} сек)")
print(f"  📦 Модели: {len(saved_models)} компонентов")
for m in saved_models:
    print(f"     ✅ {m}")
print(f"  📁 Архив: {archive_file} ({size_mb:.1f} MB)")
print(f"  💾 Drive: {DRIVE_DIR}")
print(f"{'=' * 65}")
print(f"\n  🎯 ТАРС v3 ПОЛНОСТЬЮ ОБУЧЕН!")
print(f"  🚀 Скопируйте models/ в проект и запустите: python launch_tars.py")

In [ ]:
# Скачать архив
from google.colab import files
files.download("/content/tars_v3_full.tar.gz")